In [ ]:
!pip install tritonclient[all]

## 1: Imports

In [1]:
import os
import time
from pathlib import Path
import numpy as np
import cv2
import tritonclient.http as httpclient

## 2: Setup Triton Client & Config

In [2]:
TRITON_URL = "10.67.32.50:8000"
MODEL_NAME = "ddcolor_trt"
BATCH_SIZE = 8
IMAGE_SIZE = (512, 512)

client = httpclient.InferenceServerClient(url=TRITON_URL)
if client.is_model_ready(MODEL_NAME):
    print(f"✅ Connected to Triton and model '{MODEL_NAME}' is ready.")
else:
    raise RuntimeError("❌ Model not ready.")

✅ Connected to Triton and model 'ddcolor_trt' is ready.


## 3: Define Preprocessing & Postprocessing Function

In [3]:
def preprocess_image(img_path, image_size=(512, 512)):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Cannot read {img_path}")

    orig_size = (img.shape[1], img.shape[0])
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    lab = cv2.cvtColor(rgb, cv2.COLOR_RGB2LAB)
    l = lab[:, :, 0:1]
    gray_lab = np.concatenate([l, np.zeros_like(l), np.zeros_like(l)], axis=-1)
    gray_rgb = cv2.cvtColor(gray_lab, cv2.COLOR_LAB2RGB)
    gray_rgb = cv2.resize(gray_rgb, image_size)
    inp = gray_rgb.transpose(2, 0, 1).astype(np.float32)
    return inp, l, orig_size



def postprocess_image(output_ab, orig_l, orig_size):
    W, H = orig_size
    ab = cv2.resize(output_ab.transpose(1, 2, 0), (W, H))
    lab_out = np.concatenate([orig_l, ab], axis=-1)
    rgb = cv2.cvtColor(lab_out, cv2.COLOR_LAB2RGB)
    rgb = np.clip(rgb, 0, 1)
    return (rgb * 255).astype(np.uint8)


## 4: Inference

In [4]:
def infer_batch(batch_array):
    inputs = [httpclient.InferInput("input", batch_array.shape, "FP32")]
    inputs[0].set_data_from_numpy(batch_array)
    outputs = [httpclient.InferRequestedOutput("output")]
    result = client.infer(model_name=MODEL_NAME, inputs=inputs, outputs=outputs)
    out_array = result.as_numpy("output")
    return out_array

In [5]:
path = './grayscale_images/Audrey Hepburn.jpg'
inp, l, s = preprocess_image(str(path), IMAGE_SIZE)

img_gray_rgb = np.expand_dims(inp, axis=0)
img_gray_rgb = np.concatenate([img_gray_rgb for _ in range(BATCH_SIZE)], axis=0)

In [6]:
import tqdm
times = []
for i in tqdm.tqdm(range(10)):
    start = time.time()
    output = infer_batch(img_gray_rgb)
    times.append(time.time() - start)

print(f"Average time per batch: {np.mean(times):.4f} seconds")
print(f"Average time per image: {np.mean(times)/BATCH_SIZE:.4f} seconds")
print(f"Average FPS per image: {BATCH_SIZE/np.mean(times):.4f}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.88it/s]

Average time per batch: 0.1684 seconds
Average time per image: 0.0210 seconds
Average FPS per image: 47.5140
